# Credit Repayment Behaviour Analysis

The goal to find out whether the factors such as the marital status and the number of children of the borrowers affect the repayment of the loan to the bank on time. The Bank X has provided the data on clients' creditworthiness.

The report will be considered when building a credit scoring of a potential client of the bank. A credit scoring is used to evaluate the risks of the potential borrowers defaulting on their debt obligations.

# Data Description

***Note:*** the data provided by the Bank X is about clients in Russia, thus some contents of the dataset might be in Russian language.

children : the number of children in the family
<br>days_employed: how long the customer has worked
<br>dob_years: the customer’s age
<br>education: the customer’s education level
<br>education_id: identifier for the customer’s education
<br>family_status: the customer’s marital status
<br>family_status_id: identifier for the customer’s marital status
<br>gender: the customer’s gender
<br>income_type: the customer’s income type
<br>debt: whether the client has ever defaulted on a loan
<br>total_income: monthly income
<br>purpose: reason for taking out a loan

## Step 1. Data quality assessment

In [10]:
# importing Pandas library
import pandas as pd 

In [11]:
# opening the data file
df = pd.read_csv('clients_data.csv')

In [12]:
# getting the first 5 table strings.
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [13]:
# looking at the general information
df.info()
# checking numeric values
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [14]:
# checking for at the NaN values
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Summary:** We have a table with the total of 21525 rows and 12 columns of clients' data. Data types include float64, int64, and object. 
Findings:

1. 'total_income', 'days_employed' each have 2174 NaN values. 
2. 'days_employed': has negative and very big values, such as '-18388.949901' and '401755.400475'. The data in this column doesn't make sense as it is meant to represent the number of days of the employment. It is necessary to request more information about this data.
3. 'children': the number of children '-1' and '20'
4. 'dob_years': some of the client's data indicate their age as '0' years
5. 'education', 'education_id': there are 15 unique values in 'education' and 5 in 'education_id'
6. 'gender': 3 unique values: female, male, and N/A

Possible reasons for missing values & errors in the data above: In this case, it might be the concealment of the personal information by the client (about the employment and income level). Another possibility is the data entry errors.

## Step 2. Data preprocessing

### Processing the missing values

In [15]:
#Создание "словаря" с медианой 'total_income', сгруп. по категориям из 'income_type'
income_dict = dict(df.groupby('income_type')['total_income'].median())

#Заменяем пропущенные значения в столбце 'total_income' значениями из словаря 'income_dict'
df['total_income'] = df['total_income'].fillna(df['income_type'].apply(lambda x: income_dict.get(x)))
     #Check:
#print(df.head(15))
#df.info()

#Делаем то же самое для столбца days_employed, но без словаря, т.к. кол-во дней занятости не зависит от других параметров в таблице
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
     #Check:
#print(df.head(15))
#df.info()

**Вывод** : Пропуски в датасете заполнены медианой, так как мы не знаем точные данные и не можем проверить причину их отсутствия, а удалять их из таблицы - подвергать себя потере ценной информации

1. Принцип заполнения пропусков: в пропусках типа количественных переменных бывает большая разница в значениях. В таком случае, среднее арифметическое может некорректно характеризовать данные, поэтому наибольный эффективный способ - это заполнить пропуски медианой, сперва отсортировав числа по-порядку и разделив их по категориям из столбца income_type (тип занятости), т.к. ур зп может сильно отличаться у разных должностей.
2. Так как days_employed не явл. обязательным параметром для достижения целей поставленной задачи банком, и так как их не имеет смысла сгруппировывать с другими столбцами, я заполнила этот столбец средним статистическим значением по числам в данном столбце, чтобы выровнять таблицу. 

### Замена типа данных

In [16]:
#Заменяем вещественный тип данных на целочисленный
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
#Check
#df.info()

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
А можно было так

```python
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')
```

**Вывод** : Замена вещественного типа данных на целочисленный не обязательна, но желательна в виду удобства и наглядности

1. Вещественынй тип данных - тип float. Всего в данных 2 столбца, которые содержат entries данного типа:  'days_employed' (трудовой стаж в днях) и 'total_income' (доход в месяц). Заменяем значения на целочисленные для краткости и наглядности.
2. Для изменения типа данных был использован метод astype из-за его краткости и простоты использования

### Обработка дубликатов

In [17]:
#Сначала приведем столбцы с данными типа str в нижний регистр
df['education'] = df['education'].str.lower()
#Проверка на наличие разницы в регистрах в других столбцах (не найдено - продолжаем)
#print(df['insert_column'].unique())

#Подсчет кол-ва дубликатов в датафрейме(71)
#df.duplicated().sum()

#Удаление дубликатов + проверка
df = df.drop_duplicates().reset_index(drop= True)
#df.duplicated().sum()

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Старайся промежуточные выводы/комментарии оформлять также в ячейках `Markdown`, это повышает читаемость кода. 

И оставляй решение типа `df.duplicated().sum()`, чтобы заказчику тоже все эти данные были видны

**Вывод** : Найдены и устранены дубликаты в датасете

1. Для поиска и удаления дубликатов я использовала четыре метода: первый - найти уникальные значения в столбцах str, благодаря чему обнаружилось, что есть слова с одним значением, но разного регистра, которые считаются "уникальными"(это мешает поиску дубликатов), второй - исп. str.lower() метод для Pandas, чтобы привести строки в единый нижний регистр, третий - найти кол-во дубликатов двойным методом duplicated().sum(), четвертый - удалить дубликаты методом drop_duplicates.
2. Причины появления дубликатов. Их может быть много, например человеческий фактор (некорректное введение, ошибки при вводе данных, сокрытие информации, и т.д.) или технические ошибки, например неправильный свод данных их разных источников или баг

### Лемматизация

In [18]:
from pymystem3 import Mystem
m = Mystem()

#Найдем уникальные лемматизированные значения в столбце 'purpose', чтобы использовать его в функции
lemmas = []
for words in df['purpose'].unique():
        lemmas_word = m.lemmatize(words)
        lemmas += lemmas_word
myset = list(set(lemmas))

#Так как список маленький, очистим его от пробелов, предлогов, прочего мусора вручную, оствив только описательные существительные
myset_clean = ['семья', 'образование', 'ремонт', 'покупка', 'строительство', 'автомобиль', 'жилье', 'операция', 'недвижимость', 'свадьба']

#Функция для лемматезации текста в столбце
def lemmatize_col(row):
    for word in m.lemmatize(row):
        if word in myset_clean:
            return word  
        
#Применение функции к столбцу в датафрейме
df['group_purpose'] = df['purpose'].apply(lemmatize_col)

#Проверка на наличие None
#df.isna().sum()

ModuleNotFoundError: No module named 'pymystem3'

<div class="alert alert-success"> 
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Реализация правильная, при этом предлагаю более короткое и быстрое решение с использованием [join()](https://stackoverflow.com/questions/5618878/how-to-convert-list-to-string)

    lemmas = m.lemmatize(' '.join(df['purpose']))
    Counter(lemmas)

</div>

**Вывод** : Лемматезация необходима, чтобы было удобно группировать и искать нужные значения, так как респонденты могли ввести одно и то же значение разными формулировками (прим. покупка автомобиля и приобретение автомобиля)

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

А вот моя реализация, так, для общего развития. Кстати, операция здесь идет не как медицинская услуга, а как действие с недвижимостью или жильем

In [ ]:
purposes_keys = {'жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба'}

def get_purpose(data):
    
    """Присваивает строке категорию цели"""
    
    intersection = list(purposes_keys & set(m.lemmatize(data['purpose'])))
    
    if not intersection:
        return 'категория не определена'
    return intersection[0]

df_example = pd.read_csv('/datasets/data.csv').head()

df_example.apply(get_purpose, axis=1)

0          жилье
1     автомобиль
2          жилье
3    образование
4        свадьба
dtype: object

1. Процесс лемматизации подробно описан в закомментированных (#) строках в коде выше. Note, что в ручную такая обработка myset возможна только если список получается маленьким. Если же нет, я бы воспользовалась NLTK и сделала еще один loop по 'myset', где он выбирет из списка слова которые == ‘NN’（noun).
2. словарь для лемматезации был собран из уникальных значение самого лемматизируемого столбца, чтобы увеличить точность при анализе данных

### Категоризация данных

In [ ]:
# Проверка на кол-во респондентов по кол-ву детей в семье
#print(df['children'].value_counts())

#Замена значений 20 и -1 на релевантные значения в таблице (по логике: 20 - имелось в виду 2, (-1) - имелось в виду 0)
df['children'] = df['children'].mask(df['children']==20, 2) 
df['children'] = df['children'].mask(df['children']==(-1), 0) 

#print(df['family_status'].unique())

#Сборка всех данных в пивот таблицу и суммирование значение в 'TOTAL' для наглядности
data_pivot = df.pivot_table(index=['family_status'], columns='children', values='debt', aggfunc='sum',fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot)

children                    0      1      2     3    4    5  TOTAL
family_status                                                     
Не женат / не замужем   210.0   52.0   10.0   1.0  1.0  N/A    274
в разводе                55.0   21.0    8.0   1.0  0.0  N/A     85
вдовец / вдова           53.0    7.0    3.0   0.0  0.0  N/A     63
гражданский брак        229.0  118.0   33.0   8.0  0.0    0    388
женат / замужем         517.0  246.0  148.0  17.0  3.0    0    931
TOTAL                  1064.0  444.0  202.0  27.0  4.0    0   1741


<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Для замены значений можно воспользоваться методом [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html)

</div>

**Вывод** : При просмотре данных о кол-ве детей у респондента выяснилось, что всего в таблице пришлось 76 значений на 20 детей и 47 значений на -1, а все остальные распределены между 0-5 детей. Из этого я сделала вывод, что при вводе данных произошла ошибка, и те, кто написали 20 - дописали лишний 0, а при -1 подразумевалось 0. Спросить о достоверности данного вывода нет возможности, поэтому вместо удаления данных строк, я воспользовалась данной логикой при замене. В итоге, я собрала интересующие нас числа в PivotTable.

1. Основная задача - разобраться, влияет ли семейное положение ('family_status') и количество детей ('children') клиента на факт возврата кредита в срок ('debt'). Процес категоризацыы описан в самом задании в закомментированных (#) строках в коде.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#См. Markdown
#Создадим новую Pivot Table, которая отражает количество должников с разным кол-вом детей в числах и процентах

data_pivot_new = df.pivot_table(index = 'children',  
                                values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_pivot_new.columns=['Total entries', 'Total debtors', '% of debtors']
data_pivot_new.style.format({'% of debtors': '{:.1%}'})

,Total entries,Total debtors,% of debtors
children,,,
0,14138,1064,7.5%
1,4808,444,9.2%
2,2128,202,9.5%
3,330,27,8.2%
4,41,4,9.8%
5,9,0,0.0%


**Вывод**

Не обязательно. Из таблицы понятно, что у большинства опрошенных (14 138 людей) детей нет, а должников среди них только 1064. Наибольший процент должников у категории людей с 4мя детьми(9.8% или 4 человека), но среди них был только 41 респодент. Интересно, что при этом у категории людей с 5 детьми (всего 9 человек) нет долгов. Из данных результатов я бы сделала вывод, что на % должников влияют не дети или не только дети, и важно рассмотреть также другие факторы. Тип займа в банке тоже имеет значение.

<div class="alert alert-danger">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Здесь ты посчитала абсолютные значения, однако они не в полной мере отражают характеристики этих групп. Возможно, что общее кол-во людей с детьми намного больше людей без детей, поэтому и число задолжников больше. Сразу и не скажешь. Тебе необходимо посчитать относительные показатели, т.е какой процент людей с детьми невозращают кредит относительно общего кол-ва людей этой группы (комментарий общий, относится ко всему 3 разделу). Поэтому я предлагаю выполнить следующее:
    
- построй сводные таблицы по созданным столбцам и посчитай в них средние значения по столбцу `debt`. Это тоже самое что и поделить кол-во должников (операция sum) на общее кол-во клиентов (count). Этот показатель как раз и будет демонстрировать процент должников этих групп. 
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера v2 <a class="tocSkip"> </h2>

Там ты считаешь процент пользователей определенной группы от общего кол-ва пользователей. Это немного не то, чтобы посчитать кол-во должников, можно выполнить следующее (Обрати внимание на [style](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) библиотеки `pandas`. Там много прикольных штук для того, чтобы сделать датафрейм красивее)

In [ ]:
df_example = df.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14138,1064,7.53%,92.47%
1,4808,444,9.23%,90.77%
2,2128,202,9.49%,90.51%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#См. Markdown

**Вывод**

Да. Исходя из PivotTable в шаге 2.5, и результатам TOTAL по строкам (то есть, не важно, сколько детей у человека, смотрим только на семейное положение), можно сделать вывод, что кол-во женатых / замужних с долгами больше, значительно больше, чем у всех остальных, однако у не женатых / не замужних долгов больше, чем у тех, кто в разводе, вдова/вдовец или в гражданском браке. 

|family_status|TOTAL |  
|---|---|
|Не женат / не замужем |  **274**|
|в разводе     |         **85**|
|вдовец / вдова     |     **63**|
|гражданский брак |       **388**|
|женат / замужем    |     **931**|

<div style="border:solid purple 5px; padding: 20px"> 
<h2 align="center"> Рубрика «Питонячий лайфхакер» <a class="tocSkip"> </h2>

<h3> Функция zip <a class="tocSkip"> </h3>

Функция zip создаёт итератор, который комбинирует элементы нескольких списков. Это позволяет осуществлять параллельный обход списков в циклах for или, например, выполнять параллельную сортировку.

![](https://i.ibb.co/MPPZ6TL/image.png)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#По официальным данным от Росстата, среднемесячная ЗП населения России в 2020 51083. 
#Для того чтобы считаться средним классом, в Москве необходимо зарабатывать от 121 тыс. руб

#Посчитаем, что средний ур. ЗП это приблизительно 51000-121000тыс руб. Ниже 51к - ниже ср., Выше 120к - выше ср.
def total_income_category(value):
    if value < 51000:
        return 0
    elif 51000 <= value <= 121000:
        return 1
    return 2 
df['income_level_index'] = df['total_income'].apply(total_income_category)

#Сделаем еще 1 PivoTable, разделив должников по гендерному признаку:
data_pivot = df.pivot_table(index=['income_level_index'], columns=['gender'], values='debt', aggfunc=['sum'],fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot)



                      sum                  
gender                  F      M  XNA TOTAL
income_level_index                         
0                    18.0    8.0  N/A    26
1                   386.0  186.0  N/A   572
2                   590.0  553.0    0  1143
TOTAL               994.0  747.0    0  1741


<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Подкрепление выводов внешними фактами 👍. Усиливает ощущение хорошо сделанной работы

</div>

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Тебя не смутило значение `XNA` в данных?

<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

Здесь, кстати для деления по доходу можно было применить метод [cut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) или [qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)(в этом случае деление будет проводиться по проценилям). Обрати внимание, у данного метода есть параметр `labels`, так можно оформить группы красивее 😊

</div>

**Вывод**

Из Pivot Table выше можно сделать следующие выводы: 

1. Кол-во людей с ур. дохода выше среднего (индекс 2) гораздо больше тех, у кого ур. дохода ниже (1 и 0). Всего есть 6 должников с уровнем дохода ниже среднего (индекс 0)
                 
|income_level_index|TOTAL|                       
|---|---|
|0|**6**|
|1|**572**|
|2|**1143**|

2. Дополнительно, так как я отсортировала должников по гендерному признаку, можно сделать вывод, что кол-во должников женщин с доходом выше среднего, чем у мужчин с доходом выше среднего. То же самое относится к с реднему и ниже среднего уровням дохода. 

|income_level_index|Female|Male|XNA  
|---|---|---|---|                         
|0|18.0|8.0|N/A  
|1|386.0|186.0|N/A   
|2|590.0|553.0|0   
|**TOTAL**|**994.0**|**747.0**|**0**   


<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

В Markdown редко таблицы визуализируют, видишь, в твоей реализации они получаются совсем нечитабельными. Почитай вот [здесь](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd), там есть примеры как сделать таблицы и в целом очень круто расписаны Markdown, и еще вот такую [шпаргалку](https://paulradzkov.com/2014/markdown_cheatsheet/) оставлю, вдруг понадобится 🙂
    
</div>

<font color='blue'>***Вероника***: Спасибо!</font> 

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#Создадим еще 1 PivotTable
data_pivot = df.pivot_table(index=['group_purpose'], columns='gender', values='debt', aggfunc='sum',fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot[['F','M', 'TOTAL']])

gender             F      M  TOTAL
group_purpose                     
автомобиль     172.0  105.0    277
жилье           31.0   15.0     46
недвижимость    23.0   19.0     42
образование    207.0  163.0    370
операция       107.0   98.0    205
покупка        248.0  188.0    436
ремонт          23.0   12.0     35
свадьба        108.0   78.0    186
строительство   75.0   69.0    144
TOTAL          994.0  747.0   1741


**Вывод**

Из Pivot Table выше можно сделать следующие выводы: 
1. Больше всего должников среди двух имеющихся гендеров - те, кто берут кредит с целью покупки чего-либо (спецефически женщин должников больше, чем мужчин). На 2 месте среди целей для кредита - образование, на 3 - автомобиль. Во всех случаях кол-во должников женщин больше, чем мужчин.
group_purpose  TOTAL                        
автомобиль     277
жилье          46
недвижимость   42
образование    370
операция       205
покупка        436
ремонт         35
свадьба        186
строительство  144


## Шаг 4. Общий вывод

Разберем итоги на несколько этапов:

1)После общей оценки состояния датасетя, были выявлены несколько очевидных проблем: шибки в данных, разные регистры итд. 2. Возможные причины появления пропусков: чаще всего причиной является человеческий фактор. В данном случае, это может быть сокрытие личной информации (о занятости и уровне дохода). Другой возмодной причиной могут быть ошибки ввода данных.

2)Пропуски в столбце 'total_income' были заполнены медианой после разделения по группам по доходу('income_type'), т.к. есть среди данных есть неравномерное распределение среди небольших и крупных значений, что может повлять на точность анализа. Столбец 'days_employed' был заполнен этот столбец средним статистическим значением в пропусках, чтобы выровнять датасет. Из-за ошибки формате/вводе данных в данном столбце, он не использовался при дальнейем анализе.

3) Был заменен вещественынй тип данных - тип float - на целочисленный. Всего в данных 2 столбца, которые содержат entries данного типа: 'days_employed' (трудовой стаж в днях) и 'total_income' (доход в месяц). Заменяем значения на целочисленные для краткости и наглядности.

4)Для поиска и удаления дубликатов было использовано четыре метода: первый - найти уникальные значения в столбцах str, благодаря чему обнаружилось, что есть слова с одним значением, но разного регистра, которые считаются "уникальными"(это мешает поиску дубликатов), второй - исп. str.lower() метод для Pandas, чтобы привести строки в единый нижний регистр, третий - найти кол-во дубликатов двойным методом duplicated().sum(), четвертый - удалить дубликаты методом drop_duplicates.

Причин появления дубликатов может быть много, например: человеческий фактор (некорректное введение, ошибки при вводе данных, сокрытие информации, и т.д.) или технические ошибки, например неправильный свод данных их разных источников или баг

5)При просмотре данных о кол-ве детей у кадого респондента выяснилось, что всего в таблице пришлось 76 значений на 20 детей и 47 значений на -1, а все остальные распределены между 0-5 детей. Из этого я сделала вывод, что при вводе данных произошла ошибка, и те, кто написали 20 - дописали лишний 0, а при -1 подразумевалось 0. Спросить о достоверности данного вывода нет возможности, поэтому вместо удаления данных строк, я воспользовалась данной логикой при замене. В итоге, я собрала интересующие нас числа в PivotTable

***ИТОГ***: Изначальной поставленной задачей было выяснить влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок. На анализе имеющихся данных можно сделать вывод, что наличие детей не обязательно является фактором влияния на просрочки платежей в банке,% должников в каждой группе разделенной по кол-ву детей примерно около 9% (за исключении группы с 5ю детьми). При этом, у женатых/ замужних без детей должников более чем в 2 раза больше, чем у любых других категорий. 

|family_status/children|0|1|2|3|4|5|  
|---|---|---|---|---|---|---|
|Не женат / не замужем|210.0|52.0|10.0 |1.0|1.0|N/A   |
|в разводе | 55.0 |21.0  |  8.0|   1.0 | 0.0|  N/A    |
|вдовец / вдова   |       53.0   | 7.0  |  3.0 |  0.0 | 0.0|  N/A|     
|гражданский брак   |     229.0 | 118.0  | 33.0|  8.0 | 0.0 |   0 |   
|женат / замужем   |      517.0  |246.0 | 148.0  |17.0|  3.0  |  0   |

Однако делать вывод, что не стоит выдавать кредит замужним / женатым женщинам или мужчинам если у них нет детей - неправильно. Нужен гораздо более глубокий анализ данных с учетом множества других факторов, которые могут повлиять на такие результаты. Например, у пары супругов нет детей, из-за проблем со здорвьем. 

**Рекоммендации**: 1) Уточнить формат данных в столбце 'days_employed' и перевести числа в корректную форму. 
2) необходим более повторный анализ данных с учетом множества других факторов, которые помогут ответить на вопрос влияет ли кол-во детей на наличие просрочек выплаты в банк. Конкретно, добавить в опрос несколько более подробных вопросов, прим.: 1) Есть ли в семье люди с инвалидностью/тяжелой болезнью? 2) Имеются ли долги в других банках/источниках? 3) Какие в среднем расходы на ребенка в месяц? итд

<div style="border:solid purple 2px; padding: 20px"> 

У тебя получилась очень хорошая работа! Многие моменты сделаны просто прекрасно. Ты показываешь отличное владение изучаемым материалом: уверенно пользуешься pandas, умеешь подготавливать, очищать, обогощать данные. Остались небольшие доработки:

- По всему проекту необходимо оставить и раскомментировать промежуточные решения
- В шаге 3 необходимо получить результаты с помощью сводных таблиц с расчетом относительных показателй по группам

Жду твоих исправлений, у тебя все получится 😊

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.